In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.ExcelFile('../data/datos.xlsx')
df = data.parse('Consumo')  

df_cleaned = df.dropna(how='all')

tablas_config = {
    'Cocaína': {
        'start_col': 0,
        'cols': ['País', 'Cocaína', 'Categoría_Cocaína']
    },
    'Cannabis': {
        'start_col': 4,
        'cols': ['País', 'Cannabis', 'Categoría_Cannabis']
    },
    'Metanfetaminas': {
        'start_col': 8,
        'cols': ['País', 'Metanfetaminas', 'Categoría_Metanfetaminas']
    },
    'Opioides': {
        'start_col': 12,
        'cols': ['País', 'Opioides', 'Categoría_Opioides']
    }
}

def procesar_tabla(df, start_col, nombres_columnas):
    """Procesa cada tabla individual respetando tu estructura real"""
    try:
        tabla = df.iloc[2:, start_col:start_col+3].copy()
        tabla.columns = nombres_columnas
        
        tabla = tabla.dropna(subset=['País'])
        col_valor = nombres_columnas[1]
        tabla[col_valor] = pd.to_numeric(tabla[col_valor], errors='coerce')
        
        return tabla[['País', col_valor]]
    
    except Exception as e:
        print(f"Error procesando {nombres_columnas[1]}: {str(e)}")
        return pd.DataFrame()

dataframes = {}
for droga, config in tablas_config.items():
    temp_df = procesar_tabla(df_cleaned, 
                           config['start_col'], 
                           config['cols'])
    dataframes[droga] = temp_df

df_final = dataframes['Cocaína']
for droga in ['Cannabis', 'Metanfetaminas', 'Opioides']:
    df_final = df_final.merge(dataframes[droga], 
                            on='País', 
                            how='outer')

def aplicar_categorias(df):
    categorizadores = {
        'Cocaína': [
            (1.2, 'Alta'),
            (0.6, 'Media'),
            (0.29, 'Baja'),
            (-np.inf, 'Muy baja')
        ],
        'Cannabis': [
            (6.8, 'Alta'),
            (3.9, 'Media'),
            (2.3, 'Baja'),
            (-np.inf, 'Muy baja')
        ],
        'Metanfetaminas': [
            (0.9, 'Alta'),
            (0.48, 'Media'),
            (0.17, 'Baja'),
            (-np.inf, 'Muy Baja')
        ],
        'Opioides': [
            (1, 'Alta'),
            (0.5, 'Media'),
            (0.26, 'Baja'),
            (-np.inf, 'Muy baja')
        ]
    }
    
    for droga, limites in categorizadores.items():
        col_categoria = f'Categoría_{droga}'
        df[col_categoria] = df[droga].apply(
            lambda x: next((cat for lim, cat in limites if x > lim), 'Sin dato')
        )
    
    return df

df_final = aplicar_categorias(df_final)

column_order = ['País'] + \
    [f'{d}|Categoría_{d}' for d in tablas_config.keys()]
    
df_final = df_final.fillna(0).sort_values('País').reset_index(drop=True)

In [3]:
df_final.head()

,País,Cocaína,Cannabis,Metanfetaminas,Opioides,Categoría_Cocaína,Categoría_Cannabis,Categoría_Metanfetaminas,Categoría_Opioides
0,Albania,2.50,5.60,0.10,0.00,Alta,Media,Muy Baja,Sin dato
1,Alemania,1.03,6.62,1.15,0.36,Media,Media,Alta,Baja
2,Argelia,0.01,0.52,0.03,0.00,Muy baja,Muy baja,Muy Baja,Sin dato
3,Argentina,1.20,5.67,0.00,0.19,Media,Media,Sin dato,Muy baja
4,Australia,0.00,10.80,1.73,3.10,Sin dato,Alta,Alta,Alta


In [29]:
df_final.shape

(83, 9)

In [7]:
# Leer la pestaña de Producción
df_produccion = data.parse('Producción', header=None)

# Limpiar filas completamente vacías
df_prod_cleaned = df_produccion.dropna(how='all')

# Función para procesar tablas complejas
def procesar_tabla_compleja(df, start_row, start_col, columnas):
    tabla = df.iloc[start_row:, start_col:start_col+3].copy()
    tabla.columns = columnas
    tabla = tabla.dropna(subset=[columnas[0]])
    tabla[columnas[1]] = pd.to_numeric(tabla[columnas[1]], errors='coerce')
    return tabla

# =================================================================
# 1. PROCESAR SUPERFICIE CULTIVADA
# =================================================================
cultivo_config = {
    'start_row': 4,  # Fila donde empiezan los datos reales
    'start_col': 0,
    'columnas': ['País', 'Coca', 'Opio']
}

# Procesar tabla de cultivos
df_cultivo = procesar_tabla_compleja(
    df_prod_cleaned,
    cultivo_config['start_row'],
    cultivo_config['start_col'],
    cultivo_config['columnas']
)

# Limpiar y convertir valores
df_cultivo = df_cultivo[df_cultivo['País'].str.strip().ne('')]
df_cultivo[['Coca', 'Opio']] = df_cultivo[['Coca', 'Opio']].apply(lambda x: pd.to_numeric(x.astype(str).str.replace('.', '').str.replace(',', '.'), errors='coerce'))

# =================================================================
# 2. PROCESAR MANUFACTURA
# =================================================================
manufactura_config = {
    'laboratorios': [
        {
            'nombre': 'Cocaína',
            'start_row': 2,
            'start_col': 2,
            'columnas': ['País', 'Laboratorios_Cocaina', 'Categoría_Cocaina']
        },
        {
            'nombre': 'Opioides',
            'start_row': 2,
            'start_col': 6,
            'columnas': ['País', 'Laboratorios_Opioides', 'Categoría_Opioides']
        },
        {
            'nombre': 'Metanfetaminas',
            'start_row': 2,
            'start_col': 10,
            'columnas': ['País', 'Laboratorios_Metanfetaminas', 'Categoría_Metanfetaminas']
        }
    ]
}

# Procesar cada tipo de manufactura
dfs_manufactura = []
for config in manufactura_config['laboratorios']:
    df_temp = procesar_tabla_compleja(
        df_prod_cleaned,
        config['start_row'],
        config['start_col'],
        config['columnas']
    )
    
    # Limpieza específica para manufactura
    df_temp = df_temp[~df_temp.iloc[:, 1].astype(str).str.contains('NaN|^\s*$')]
    df_temp['País'] = df_temp['País'].str.strip()
    
    dfs_manufactura.append(df_temp)

# Combinar todas las manufacturas
df_manufactura = dfs_manufactura[0]
for df in dfs_manufactura[1:]:
    df_manufactura = df_manufactura.merge(df, on='País', how='outer')

# Estandarizar categorías
categorias_map = {
    'ALTA': 'Alta',
    'MEDIA': 'Media',
    'BAJA': 'Baja',
    'MUY BAJA': 'Muy baja',
    'MUY BAJO': 'Muy baja'
}

for col in df_manufactura.filter(like='Categoría').columns:
    df_manufactura[col] = df_manufactura[col].str.strip().replace(categorias_map).fillna('Sin dato')

# =================================================================
# RESULTADOS FINALES
# =================================================================


DataFrame de Cultivos:
           País          Coca          Opio
10     Bolivia   2.600000e+05           NaN
11    Colombia   1.197948e+07  4.087556e+14
12        Peru   5.849921e+14           NaN
13   Afganistán           NaN  1.874143e+14
14      Myanmar           NaN  4.322308e+14

DataFrame de Manufactura:
                                                País  Laboratorios_Cocaina  \
0                                               ALTA                   NaN   
1                                               BAJA                   NaN   
2                                              MEDIA                   NaN   
3                                           MUY BAJA                   NaN   
4  Superficie de cultivos de opio por país. Hecta...                   NaN   

  Categoría_Cocaina  Laboratorios_Opioides Categoría_Opioides  \
0          Sin dato                    NaN           Malaysia   
1          Sin dato                    NaN       Países Bajos   
2          Sin dato   

In [8]:
df_cultivo.head()

,País,Coca,Opio
10,Bolivia,2.600000e+05,NaN
11,Colombia,1.197948e+07,4.087556e+14
12,Peru,5.849921e+14,NaN
13,Afganistán,NaN,1.874143e+14
14,Myanmar,NaN,4.322308e+14


In [9]:
df_manufactura.head()

,País,Laboratorios_Cocaina,Categoría_Cocaina,Laboratorios_Opioides,Categoría_Opioides,Laboratorios_Metanfetaminas,Categoría_Metanfetaminas
0,ALTA,NaN,Sin dato,NaN,Malaysia,NaN,Estados Unidos
1,BAJA,NaN,Sin dato,NaN,Países Bajos,NaN,Eslovaquia
2,MEDIA,NaN,Sin dato,NaN,Nueva Zelandia,NaN,New Zealand
3,MUY BAJA,NaN,Sin dato,NaN,Estonia,NaN,Sudáfrica
4,Superficie de cultivos de opio por país. Hecta...,NaN,Sin dato,NaN,Sin dato,NaN,Sin dato


In [12]:
# Leer la pestaña de Tráfico
df_trafico = data.parse('Tráfico', header=None)

# Limpiar filas completamente vacías
df_trafico_clean = df_trafico.dropna(how='all')

# Identificar fila de encabezados (corregido)
header_row = 1  # La fila REAL con los encabezados es la 1 (índice 1)
df_trafico_clean.columns = ['Col_extra', 'País', 'Cocaína', 'Cannabis', 'Metanfetaminas', 'Heroína']

# Eliminar columna adicional vacía
df_trafico_clean = df_trafico_clean.drop(columns=['Col_extra'])

# Procesar desde la fila de datos (índice 2 en adelante)
df_trafico_final = df_trafico_clean.iloc[2:].copy()

# Reiniciar índices y limpiar
df_trafico_final = df_trafico_final.reset_index(drop=True)
df_trafico_final['País'] = df_trafico_final['País'].str.strip()

# Función mejorada de limpieza
def limpiar_valores(valor):
    if isinstance(valor, str):
        try:
            return float(valor.replace('$', '').replace(',', '').strip())
        except:
            return np.nan
    return valor

# Aplicar a todas las columnas numéricas
for col in ['Cocaína', 'Cannabis', 'Metanfetaminas', 'Heroína']:
    df_trafico_final[col] = df_trafico_final[col].apply(limpiar_valores)

print("DataFrame final corregido:")
print(df_trafico_final.head(3))

                 País Población  NaN  Mercado_USD
0              100398       NaN  NaN          NaN
1             1056208       NaN  NaN          NaN
2               11000       NaN  NaN          NaN
3              112398       NaN  NaN          NaN
4              121585       NaN  NaN          NaN
..                ...       ...  ...          ...
67             Suecia       NaN  NaN     8.333034
68  Trinidad y Tobago       NaN  NaN     0.537840
69              Túnez       NaN  NaN     0.913000
70            Uruguay       NaN  NaN     5.717040
71          Venezuela       NaN  NaN    18.086862

[72 rows x 4 columns]


/var/folders/s7/6ncckdrj32j81xfl9hjxkzkr0000gn/T/ipykernel_1453/2134311689.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Reino Unido (Irlanda del Norte)': 'Reino Unido', 'Chequia': 'República Checa'})


In [13]:
df_trafico_final.head()

,País,Cocaína,Cannabis,Metanfetaminas,Heroína
0,Sri Lanka,177609.980000,1731.53,4567.113736,145917.640000
1,Australia,159530.390000,NaN,85635.359116,142964.640000
2,Finlandia,94686.555524,5751.15,14874.141876,53374.333333
3,Palestina,75000.000000,4464.29,NaN,NaN
4,Moldavia,71174.380000,NaN,NaN,126448.890000


In [29]:
df_mercado = data.parse('Estimación de Mercado', header=None)
df_mercado_clean = df_mercado.dropna(how='all', axis=0)


# Ajustar según posiciones B4, E4, H4 (columnas 1, 4, 7 en pandas)
def procesar_columna_ajustada(df, start_col):
    tabla = df.iloc[1:, start_col:start_col+2].copy()  # Filas desde B4 (índice 3)
    tabla.columns = ['País', df.iloc[3, start_col+1]]  # Encabezados en fila 3
    tabla = tabla.dropna(subset=['País'])
    return tabla.reset_index(drop=True)

# Procesar cada tabla (B=1, E=4, H=7)
poblacion = procesar_columna_ajustada(df_mercado_clean, 1).rename(
    columns={df_mercado_clean.iloc[3,2]: 'Población'})

consumo = procesar_columna_ajustada(df_mercado_clean, 4).rename(
    columns={df_mercado_clean.iloc[3,5]: 'Consumo_Tn'})

mercado = procesar_columna_ajustada(df_mercado_clean, 7).rename(
    columns={df_mercado_clean.iloc[3,8]: 'Mercado_USD'})

# Función de limpieza definitiva
def limpiar_final(df):
    df['País'] = df['País'].astype(str).str.strip()
    df = df.replace({
        'Reino Unido (Irlanda del Norte)': 'Reino Unido',
        'Chequia': 'República Checa',
        'México': 'México'  # Mantener consistencia
    })
    if 'Mercado_USD' in df.columns:
        df['Mercado_USD'] = df['Mercado_USD'].astype(str).str.replace('[$,]', '', regex=True).replace('nan', np.nan).astype(float)
    return df

# Combinar resultados
df_final = (
    limpiar_final(poblacion)
    .merge(limpiar_final(consumo), on='País', how='outer')
    .merge(limpiar_final(mercado), on='País', how='outer')
    .sort_values('Población', ascending=False)
    .reset_index(drop=True)
)

# Convertir a tipos numéricos
df_final[['Población', 'Consumo_Tn']] = df_final[['Población', 'Consumo_Tn']].apply(pd.to_numeric, errors='coerce')

/var/folders/s7/6ncckdrj32j81xfl9hjxkzkr0000gn/T/ipykernel_1453/3042164216.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({
/var/folders/s7/6ncckdrj32j81xfl9hjxkzkr0000gn/T/ipykernel_1453/3042164216.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({
/var/folders/s7/6ncckdrj32j81xfl9hjxkzkr0000gn/T/ipykernel_1453/3042164216.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_

In [30]:
df_final.head()

,País,Población,Consumo_Tn,Mercado_USD
0,Estados Unidos,7409071,614.952893,1.875606e+10
1,Brasil,4488953,372.583099,9.314577e+08
2,India,1300000,107.900000,6.438035e+09
3,España,1056208,87.665264,3.748080e+09
4,Alemania,831375,69.004125,3.039619e+09


In [31]:
df_final[df_final.País == 'Estados Unidos']

,País,Población,Consumo_Tn,Mercado_USD
0,Estados Unidos,7409071,614.952893,1.875606e+10


In [32]:
df_final.País.unique()

array(['Estados Unidos', 'Brasil', 'India', 'España', 'Alemania',
       'Francia', 'México', 'Australia', 'Italia', 'Argentina', 'Egipto',
       'Colombia', 'Ecuador', 'Venezuela', 'Polonia', 'Perú', 'Guatemala',
       'Austria', 'Bélgica', 'Suecia', 'Irlanda', 'Dinamarca', 'Uruguay',
       'Costa Rica', 'Bolivia', 'Croacia', 'Grecia', 'República Checa',
       'Reino Unido', 'Bulgaria', 'El Salvador', 'Estonia', 'Letonia',
       'Eslovenia', 'Túnez', 'Trinidad y Tobago'], dtype=object)

In [33]:
len(df_final.País)

36